# Replay  데이터 증강
### Replay 분석 및 계획
---
ID 130 뒤에는 항상 140이옴  
학습데이터 (정상, 공격상황 모두) 와 제출데이터 모두 동일하게 적용됨  
130세트의 수 길이가 크게 차이나지 않음  
최소 기준점을 ID 130으로 잡음

* 130 패턴 전체를 한 세트로 잡아야 좋을지? <br> ex) Data_7의 반복되는 패턴 0A\~0F\~0\~9 <br> 공격 데이터에서는 Data_7 은 0E로 시작하는 것만은 아님

#### 공격세트 구성안
> * 세트 구성 ID 130으로 패턴 시작을 랜덤하게 하여 한 세트를 구성 <br>
* ID 130 세트 시작점을 랜덤하게 구성
* ID 상관없이 랜덤하게 세트 구성


### 구성 계획
---
1. 학습데이터에서 세트 추출
2. 충분한 세트 수 추출 후 여러 세트로 구성한 공격 세트 제작
3. 공격세트를 주입할 원본 데이터에 삽입

### 공격데이터 삽입
1~8 사이의 랜덤한 간격으로 한 row씩 삽입

In [1]:
import pandas as pd
import time
from random import *
from tqdm import notebook

In [2]:
origin_fname = "Pro_Cybersecurity_Car_Hacking_D_training-0.csv"
target_fname = "Pro_Cybersecurity_Car_Hacking_D_training-1.csv"

In [3]:
data = pd.read_csv(origin_fname)
target = pd.read_csv(target_fname)

## Replay 공격 세트 추출 및 제작 -> 폐기(긴 실행시간)

In [ ]:
# 추출 필요 parameter: 원본추출데이터, 
# 130만날 때 마다 잘라서 리스트에 저장 -> 세트 수 리스트 완성
# 리스트 반환


def GetReplay(df):
    
    startIndex = 0
    endIndex = len(df)
    cur = startIndex
    
    setIndex = 0
    replayList = list()
    
    
    for i in tqdm(range(endIndex)):
        if str(df.loc[cur, "Arbitration_ID"]) == "130": # 처음 찾은 130 1세트 시작
            #replayList = list()
            replayList.append(df.iloc[[cur]])
            cur += 1
            while df.loc[cur, "Arbitration_ID"] != "130":
                replayList[setIndex] = pd.concat([replayList[setIndex], df.iloc[[cur]]]) # 130 한 사이클 한 세트 구성
                cur += 1
            setIndex += 1
            # return replayList
        else:
            cur += 1
        
    return replayList
        

In [ ]:
test = GetReplay(data)

In [ ]:
test[0]

## Replay 세트 시작 인덱스 추출

In [4]:
# 추출 필요 parameter: 원본추출데이터, 
# 130만날 때 마다 리스트 인덱스 추출
# 130의 인덱스 리스트 반환


def GetRepIdx(df):
    
    startIndex = 0
    endIndex = len(df)
    cur = startIndex
    
    setIndex = 0
    repIdxList = list()
    
    
    for i in notebook.tqdm(range(endIndex)):
        if str(df.loc[cur, "Arbitration_ID"]) == "130": # 처음 찾은 130 1세트 시작
            #replayList = list()
            repIdxList.append(cur)
            cur += 1
        else:
            cur += 1
        
    return repIdxList
        

In [5]:
testIdxList = GetRepIdx(data)

## 1. 공격 세트 제작 - 시작인덱스를 이용하여 랜덤하게 배치하여 세트제작

In [16]:
# 세트 제작 필요 parmeter: 데이터, 시작인덱스리스트, 공격 구성 세트 수
# 시작인덱스리스트 길이에서 난수를 뽑아 세트수 구성하여 공격세트 구성

def MakeAttackSet(df, idxList, setNum):
    endIndex = len(df)
    
    randomNum = randrange(len(idxList))
    attackSet = list() #각 세트를 리스트에 저장
    attackSetidx = 0 #각 세트를 저장한 리스트의 인덱스
    
    for i in notebook.tqdm(range(setNum), desc = "세트 리스트 생성"):
        cur = idxList[randrange(len(idxList))]
        if str(df.loc[cur, "Arbitration_ID"]) == "130":
            attackSet.append(df.iloc[[cur]])
            cur += 1
            count = 1 ############130 한세트의 갯수############
            while str(df.loc[cur, "Arbitration_ID"]) != "130":
                tmp = df.iloc[[cur]]
                attackSet[attackSetidx] = pd.concat([attackSet[attackSetidx], tmp])
                cur += 1
                
                count += 1
                
            print("set[%d] 길이: " %(attackSetidx) ,count)
            attackSetidx += 1 # 다음 세트로
            
    # 세트를 저장한 리스트를 이용하여 공격 세트 한개 제작
    for i in notebook.tqdm(range(1, setNum), desc = "세트 리스트를 이용하여 공격 세트 만드는 중"):
        attackSet[0] = pd.concat([attackSet[0], attackSet[i]])
    
    resultSet = attackSet[0].copy()
    resultSet["Class"] = "Attack"
    resultSet["SubClass"] = "Replay"
    return resultSet

In [17]:
test = MakeAttackSet(data, testIdxList, 13)

test

set[0] 길이:  26
set[1] 길이:  28
set[2] 길이:  28
set[3] 길이:  25
set[4] 길이:  19
set[5] 길이:  24
set[6] 길이:  30
set[7] 길이:  26
set[8] 길이:  24
set[9] 길이:  31
set[10] 길이:  19
set[11] 길이:  20
set[12] 길이:  20



,Timestamp,Arbitration_ID,DLC,Data_1,Data_2,Data_3,Data_4,Data_5,Data_6,Data_7,Data_8,Class,SubClass
6081,1597707830,130,8,E8,7F,B4,80,0,0,3,CA,Attack,Replay
6082,1597707830,140,8,70,81,0,6F,20,0,3,45,Attack,Replay
6083,1597707830,367,8,0,0,0,0,5,0,85,12,Attack,Replay
6084,1597707830,368,8,0,0E,0,0,2,50,16,44,Attack,Replay
6085,1597707830,387,8,79,46,82,45,0,8A,4,0,Attack,Replay
...,...,...,...,...,...,...,...,...,...,...,...,...,...
103032,1597707870,260,8,7,35,1C,30,FF,D4,BB,57,Attack,Replay
103033,1597707870,47F,8,0,4C,FF,FA,0,78,0,BE,Attack,Replay
103034,1597707870,153,8,20,80,10,FF,0,FF,80,2E,Attack,Replay
103035,1597707870,220,8,0C,4,82,0,0,E7,0F,F1,Attack,Replay


# 공격세트 삽입

In [ ]:
# 삽입 필요 parameter: 타겟데이터, 공격수, 공격 별 세트 수
# parameter에 따라 세트를 랜덤하게 뽑아 공격을 만들어 리스트에 저장
# 리스트에 있는 인덱스를 랜덤하게 뽑아 매개변수의 공격 수를 타겟데이터에 적용

# 연습장

In [ ]:
print(test)
#print(data.iloc[3])
testList = list()
testList.append(data.iloc[[3]])
testList.append(data.iloc[[4]])
testList.append(data.iloc[[4]])
#pd.DataFrame(testList)
testResultList = pd.concat([testList[0], testList[1], testList[2]])
testResultList

In [15]:
data.iloc[randrange(len(testIdxList))]
target

,Timestamp,Arbitration_ID,DLC,Data_1,Data_2,Data_3,Data_4,Data_5,Data_6,Data_7,Data_8,Class,SubClass
0,1597759710,153,8,20,A1,10,FF,0,FF,50,1F,Normal,Normal
1,1597759710,220,8,13,24,7F,60,5,FF,BF,10,Normal,Normal
2,1597759710,507,4,8,0,0,1,NaN,NaN,NaN,NaN,Normal,Normal
3,1597759710,356,8,0,0,0,80,16,0,0,0,Normal,Normal
4,1597759710,340,8,FC,3,0,E4,B7,21,FA,3C,Normal,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...
806385,1597760015,366,7,3B,28,0B,3B,30,0,1,NaN,Normal,Normal
806386,1597760015,367,8,0,0,0,0,5,0,0,0,Normal,Normal
806387,1597760015,368,8,0,0,0,0,1,28,0B,42,Normal,Normal
806388,1597760015,47F,8,4,7F,FF,FF,0,7B,0,26,Normal,Normal
